In [3]:
import cv2
import numpy as np

# Load image
img = cv2.imread('../Dataset/data_testing_system/raw_images/raw_images (2).jpg')

# Crop image to square
height, width, channels = img.shape
if height > width:
    crop_size = width
    y = int((height - width) / 2)
    x = 0
else:
    crop_size = height
    x = int((width - height) / 2)
    y = 0
cropped_image = img[y:y+crop_size, x:x+crop_size]

# Konversi gambar ke skala abu-abu
gray = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)

# Thresholding pada gambar untuk memisahkan objek dari latar belakang
_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

# Temukan kontur pada gambar yang telah dithreshold dan dapatkan kontur terbesar yang diasumsikan sebagai objek daun
contours, _ = cv2.findContours(
    thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnt = max(contours, key=cv2.contourArea)

# Dapatkan koordinat bounding box (kotak pembatas) dari kontur
x, y, w, h = cv2.boundingRect(cnt)

# Buat mask dengan ukuran yang sama dengan gambar dan inisialisasi dengan nilai 0 (hitam)
mask = np.zeros(cropped_image.shape[:2], np.uint8)

# Tentukan model latar belakang (background) dan objek (foreground)
bgdModel = np.zeros((1, 65), np.float64)
fgdModel = np.zeros((1, 65), np.float64)

# Tentukan persegi panjang (rectangle) yang mencakup objek daun (ROI)
rect = (x, y, w, h)

# Algoritma GrabCut untuk menghapus latar belakang
cv2.grabCut(cropped_image, mask, rect, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_RECT)

# Buat mask dimana piksel non-nol menunjukkan objek (foreground) yang mungkin
mask2 = np.where((mask == cv2.GC_FGD) | (
    mask == cv2.GC_PR_FGD), 255, 0).astype('uint8')

# Balikkan mask ke negatif (putih menjadi hitam dan sebaliknya)
mask2_inv = cv2.bitwise_not(mask2)

# Buat hasil segmentasi dengan latar belakang putih
result = np.zeros_like(cropped_image)
result[np.where(mask2_inv == 255)] = (255, 255, 255)
result[np.where(mask2_inv == 0)] = cropped_image[np.where(mask2_inv == 0)]

# Menggambar kotak ROI pada hasil segmentasi
cv2.rectangle(cropped_image, (x, y), (x+w, y+h), (255, 0, 0), 5)

# Menampilkan hasil threshold
thresh = cv2.resize(thresh, (512, 512))
cv2.imshow('Thresh', thresh)

# Resize image
resized_img = cv2.resize(cropped_image, (512, 512))
cv2.imshow('Original', resized_img)

# Crop gambar hasil segmentasi menggunakan ROI
roi = result[y:y+h, x:x+w]
normalized_image = cv2.normalize(roi, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)

resized_cropped = cv2.resize(normalized_image, (512, 512))
cv2.imshow('Cropped', resized_cropped)

cv2.waitKey(0)
cv2.destroyAllWindows()